# Using Jurassic-2 Large on SageMaker through Model Packages

## <span style='color:Red'>THIS MODEL HAS BEEN DEPRECATED</span>
### In its place, we recommend [Jurassic-2 Light](https://github.com/AI21Labs/SageMaker/blob/main/J2_Light_example_model_use.ipynb), an improved model that provides quality results both with natural language instructions and with few shot prompts.

===============================================

This sample notebook shows you how to deploy **Jurassic-2 Large** using Amazon SageMaker.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Interact with the model](#B.-Interact-with-the-model)
   1. [Create a few-shot prompt](#C.-Create-a-few-shot-prompt)
   1. [Perform real-time inference!](#D.-Perform-real-time-inference)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you recieved this notebook from model catalog on SageMaker AWS Management Console.

In [1]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/j2-large-v1-0-43-03eb8175aa02346c9bd81decfd2b1e56"
}

In [2]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [3]:
boto3.__version__

'1.26.74'

### Install ai21 python SDK

In [4]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [5]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [6]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [7]:
endpoint_name = "j2-large"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.48xlarge"
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

----------------

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

You can think of Jurassic-2 Large as a smart auto-completion algorithm: it is very good at latching onto hints and patterns expressed in plain English, and generating text that follows the same patterns.

These two helpful concepts are worth being familiar with:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generates.

Enter a simple prompt: "To be, or", and let the model complete it

In [32]:
response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt="To be, or",
                                   maxTokens=4,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

 not to be: that is the question


As you can see, the model identifies the beginning of a famous quote, and completes it correctly.

### C. Create a few-shot prompt

The best way to guide the model is to provide several examples of input-output pairs in the prompt. This establishes a pattern for the model to mimic. Then add the input for a query example and let the model complete it with an appropriate generation. 

In this example, we will create a prompt for performing named entity recognition (NER) on news headlines.

To do this, we will build a few-shot prompt comprised of the following:

1. Prefix with 3 examples. Each example contains the relevant input (a news headline) and the output (extraction of entity, location and time). They are separated by "##".

2. The query input. An unseen headline for which we would like the model to output the entity, location and time. This should follow the same format of the inputs in the prefix.


First, we collect some example data for the prompt prefix:

In [33]:
EXAMPLES_DATA = [
    {"headline": "Inflation cooled to 6% in February 2023 as the Federal Reserve weighs next steps on interest rates", 
     "entity": "the Federal Reserve", 
     "time": "February 2023",
     "location": "NA"},
    {"headline": "Novo Nordisk to lower list price of some of its insulin by up to 75% in the U.S.", 
     "entity": "Novo Nordisk", 
     "time": "NA",
     "location": "the U.S."},
   {"headline": "John Snow says protecting Winterfell is not a 'vital' national interest", 
     "entity": "John Snow", 
     "time": "NA",
     "location": "Winterfell"}
]

Then, we use the following helper functions to construct the prefix:

In [34]:
def make_single_example(headline, entity, time, location):
    example = "Extract from the following headline these properties: Entity, Time, Location. In the case where it doesn't appear in the sentence, write NA.\n"
    example += f"Headline: {headline}\n"
    if entity:
        example += f"Entity: {entity}\n"
        example += f"Time: {time}\n"
        example += f"Location: {location}"
    
    return example

SEPARATOR = "\n##\n"

FEW_SHOT_PREFIX = SEPARATOR.join(
    make_single_example(x["headline"], x["entity"], x["time"], x["location"]) for x in EXAMPLES_DATA
)

And finally, we create a function to handle query inputs and create the full prompt:

In [35]:
def create_ner_prompt(headline):
    """
    Create a few-shot prompt to extract named entities with Jurassic-2 Large given a headline
    The prompt contains a preset sequence of examples followed by the query headline
    """
    return FEW_SHOT_PREFIX + SEPARATOR + make_single_example(headline, '', '', '')  # keep the entities blank and let the model generate

Let's see how this looks for a a new headline about what's happening in Springfield:

In [36]:
prompt = create_ner_prompt(headline="Homer Simpson to sign executive order on January 2024 to increase the number of gun background checks in Springfield")

print(prompt)

Extract from the following headline these properties: Entity, Time, Location. In the case where it doesn't appear in the sentence, write NA.
Headline: Inflation cooled to 6% in February 2023 as the Federal Reserve weighs next steps on interest rates
Entity: the Federal Reserve
Time: February 2023
Location: NA
##
Extract from the following headline these properties: Entity, Time, Location. In the case where it doesn't appear in the sentence, write NA.
Headline: Novo Nordisk to lower list price of some of its insulin by up to 75% in the U.S.
Entity: Novo Nordisk
Time: NA
Location: the U.S.
##
Extract from the following headline these properties: Entity, Time, Location. In the case where it doesn't appear in the sentence, write NA.
Headline: John Snow says protecting Winterfell is not a 'vital' national interest
Entity: John Snow
Time: NA
Location: Winterfell
##
Extract from the following headline these properties: Entity, Time, Location. In the case where it doesn't appear in the sentenc

### D. Perform real-time inference

Time to put Jurassic-2 Large to work!

Let it extract the entity, time and location from our provided headline.

In [37]:
response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=prompt,
                                   maxTokens=30,
                                   temperature=0,
                                   stopSequences=['##'],
                                   numResults=1)

print(response['completions'][0]['data']['text'])

Entity: Homer Simpson
Time: January 2024
Location: Springfield



### Interested in learning more?
Take a look at our [blog post](https://www.ai21.com/blog/building-cv-profile-generator-using-ai21-studio) to understand the process of building a good prompt.

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [38]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [39]:
model.delete_model()